# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import copy
import pickle

import torch
from torch_geometric.data import Data

import wandb

import sbmlcore
import sbmlsim
import gumpy

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
full_dataset = torch.load('full_orig_dataset.pth')

### Set Up Params and Sweep Config

In [3]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = 10000
cutoff_distance = 6.3  

# gcn params
num_node_features = 12
batch_size = 256
hidden_channels = 64
learning_rate = 0.001
wd = 5e-5
epochs = 1000

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [4]:
# First sweep:

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-2
        },
    'dropout': {
          'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 3,
        'max': 10 
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-1
      },
    }

sweep_config['parameters'] = parameters_dict

# Second / third sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [64, 128, 192, 256, 320, 384]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 5e-7, 
        'max': 1e-2
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-2 
        },
    })

# Fourth / Fifth sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [128, 192, 256, 320]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-5,
        'max': 5e-2
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 3,
        'max': 9
        },
    })

# Sixth sweep:

parameters_dict.update({
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-7,
        'max': 5e-3
        }
    })



In [5]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'cutoff_distance': {'distribution': 'uniform',
                                    'max': 9,
                                    'min': 3},
                'dropout': {'values': [0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [128, 192, 256, 320]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.005,
                                  'min': 5e-07},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.001,
                                 'min': 1e-05}}}


### Define Training Loop

In [6]:
project = "pnca-sweep-1"

In [7]:
# sweep_id = wandb.sweep(sweep_config, project=project)
# sweep_id = '18ili8gf'
# sweep_id = 'z95xpbwd'
# sweep_id = 'tr8mwg42'
# sweep_id = '2hsfpqs6'
sweep_id = 'gt2h3xwl'

In [8]:
def sweep_run():

    with wandb.init() as run:
        config = run.config
        
        model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = config.cutoff_distance,
            edge_weight_func = '1-(dist/cutoff)',
            # edge_weight_func = 'none',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')

In [9]:
wandb.agent(sweep_id, sweep_run, project = project, count=200)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Agent Starting Run: o78hjtpy with config:
wandb: 	cutoff_distance: 6.625111673194254
wandb: 	dropout: 0.8
wandb: 	hidden_channels: 128
wandb: 	learning_rate: 0.003948819837768729
wandb: 	weight_decay: 0.0002500171724956479
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dyl

Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.7386, Test Acc: 0.7283, Train Loss: 0.5218, Test Loss: 0.5426
Epoch: 020, Train Acc: 0.4863, Test Acc: 0.5187, Train Loss: 4.6305, Test Loss: 4.3345
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 32.0698, Test Loss: 33.3199
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


Test Accuracy,▁▂▂▂▂▂▁▁▁▇▂▁▂▂▂▁▂▂▂▂▂█▂▂▂▂▂▂▁▂
Test F1,▁▇▇▇▇▂▁▇▁▇▇▁▇▇▇▂▇▇▇▂▇█▇▇▂▇▇▇▁▇
Test Loss,▁▁▁▁▁▁▁▁▂▁▁▂▂▃▃▂▂▄▂▂▂▁▅▆▂▂▃▁▃█
Test Sensitivity,▁████▁▁█▁▅█▁███▁███▁█▆██▂███▁█
Test Specificity,█▁▁▁▁██▁█▇▁█▁▁▁█▁▁▁█▁▇▁▁█▁▁▁█▁
Train Accuracy,▁▂▂▂▂▁▁▂▁▆▂▁▂▂▂▁▂▂▂▁▂█▂▂▁▂▂▂▁▂
Train F1,▁▇▇▇▇▁▁▇▁▇▇▁▇▇▇▁▇▇▇▁▇█▇▇▂▇▇▇▁▇
Train Loss,▁▁▁▁▁▁▁▁▂▁▁▂▂▃▃▂▂▄▂▂▂▁▅▆▂▂▂▁▃█
Train Sensitivity,▁████▁▁█▁▅█▁███▁███▁█▆██▁███▁█
Train Specificity,█▁▁▁▁██▁█▇▁█▁▁▁█▁▁▁█▁█▁▁█▁▁▁█▁
Test Accuracy,0.511


Traceback (most recent call last):
  File "/tmp/ipykernel_628937/44313289.py", line 28, in sweep_run
    torch.save(model, f'saved_models/carter_ds_aug/{project}/{sweep_id}/{run.name}')
  File "/home/ubuntu/.local/lib/python3.10/site-packages/torch/serialization.py", line 651, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/home/ubuntu/.local/lib/python3.10/site-packages/torch/serialization.py", line 525, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/home/ubuntu/.local/lib/python3.10/site-packages/torch/serialization.py", line 496, in __init__
    super().__init__(torch._C.PyTorchFileWriter(self.name))
RuntimeError: Parent directory saved_models/carter_ds_aug/pnca-sweep-1/gt2h3xwl does not exist.
Run o78hjtpy errored:
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_628937/44313289.py", line 28, in sweep_run


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5897, Test Acc: 0.6370, Train Loss: 0.6731, Test Loss: 0.6627
Epoch: 020, Train Acc: 0.6719, Test Acc: 0.6793, Train Loss: 0.6467, Test Loss: 0.6367
Epoch: 030, Train Acc: 0.7156, Test Acc: 0.7073, Train Loss: 0.6178, Test Loss: 0.6119
Epoch: 040, Train Acc: 0.6737, Test Acc: 0.6863, Train Loss: 0.6097, Test Loss: 0.6021
Epoch: 050, Train Acc: 0.7706, Test Acc: 0.7263, Train Loss: 0.5622, Test Loss: 0.5806
Epoch: 060, Train Acc: 0.5290, Test Acc: 0.5020, Train Loss: 0.6553, Test Loss: 0.7017
Epoch: 070, Train Acc: 0.7530, Test Acc: 0.7343, Train Loss: 0.5106, Test Loss: 0.5356
Epoch: 080, Train Acc: 0.7819, Test Acc: 0.7553, Train Loss: 0.4830, Test Loss: 0.5258
Epoch: 090, Train Acc: 0.7137, Test Acc: 0.7093, Train Loss: 0.5550, Test Loss: 0.5844
Epoch: 100, Train Acc: 0.7963, Test Acc: 0.7620, Train Loss: 0.4425, Test Loss: 0.5065
Epoch: 110, Train Acc: 0.7777, Test Acc: 0.7503, Train Loss: 0.4544

...

wandb: Sweep Agent: Waiting for job.
500 response executing GraphQL.
{"errors":[{"message":"context deadline exceeded"}]}
wandb: ERROR Error while calling W&B API: context deadline exceeded (<Response [500]>)
wandb: Job received.
wandb: Agent Starting Run: 85qiages with config:
wandb: 	cutoff_distance: 4.889425203197808
wandb: 	dropout: 0.5
wandb: 	hidden_channels: 256
wandb: 	learning_rate: 0.001525590672129621
wandb: 	weight_decay: 1.7456325614690555e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.6914, Test Acc: 0.6883, Train Loss: 0.5703, Test Loss: 0.5837
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 3.0831, Test Loss: 3.4032
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 4.9921, Test Loss: 5.4484
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


Test Accuracy,▁▂▂▆▁▅▂▁▁▆▄▂▄█▂▂▂▃▂▂▂▂▂▁▁▂▃▂▂▂▂▃▂▂
Test F1,▁▇▇▇▂▆▇▁▁▇▅▇▄█▇▇▇▃▇▇▇▇▇▁▁▇▄▃▇▇▇▄▇▇
Test Loss,▁▁▁▁▁▁▁▁▁▁▁▂▁▁▃▂▂▁▂▂▂▄▅▄▃█▂▂▂▂▃▂▄▄
Test Sensitivity,▁██▅▁▄█▁▁▄▃█▃▆███▂█████▁▁█▂▂███▂██
Test Specificity,█▁▁▇██▁████▁█▇▁▁▁█▁▁▁▁▁██▁██▁▁▁█▁▁
Train Accuracy,▁▂▂▅▁▄▂▁▁▅▃▂▃█▂▂▂▂▂▂▂▂▂▁▁▂▃▂▂▂▂▃▂▂
Train F1,▁▇▇▆▁▅▇▁▁▆▄▇▄█▇▇▇▃▇▇▇▇▇▁▁▇▃▂▇▇▇▃▇▇
Train Loss,▁▁▁▁▁▁▁▁▂▁▁▂▁▁▃▂▂▂▂▂▂▄▅▄▃█▂▂▂▂▃▂▄▄
Train Sensitivity,▁██▅▁▃█▁▁▄▃█▂▆███▂█████▁▁█▂▂███▂██
Train Specificity,█▁▁▇██▁████▁█▇▁▁▁█▁▁▁▁▁██▁██▁▁▁█▁▁
Test Accuracy,0.511


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ljekgal8 with config:
wandb: 	cutoff_distance: 5.8868582672865415
wandb: 	dropout: 0.4
wandb: 	hidden_channels: 192
wandb: 	learning_rate: 1.0528678561821133e-05
wandb: 	weight_decay: 0.00019162077149086635
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6871, Test Loss: 0.6874
Epoch: 020, Train Acc: 0.5059, Test Acc: 0.4730, Train Loss: 0.6796, Test Loss: 0.6772
Epoch: 030, Train Acc: 0.5016, Test Acc: 0.4777, Train Loss: 0.6730, Test Loss: 0.6704
Epoch: 040, Train Acc: 0.5110, Test Acc: 0.4857, Train Loss: 0.6663, Test Loss: 0.6646
Epoch: 050, Train Acc: 0.5106, Test Acc: 0.4853, Train Loss: 0.6579, Test Loss: 0.6552
Epoch: 060, Train Acc: 0.7220, Test Acc: 0.6823, Train Loss: 0.6490, Test Loss: 0.6452
Epoch: 070, Train Acc: 0.7223, Test Acc: 0.7067, Train Loss: 0.6396, Test Loss: 0.6341
Epoch: 080, Train Acc: 0.7369, Test Acc: 0.6883, Train Loss: 0.6287, Test Loss: 0.6282
Epoch: 090, Train Acc: 0.7341, Test Acc: 0.6840, Train Loss: 0.6198, Test Loss: 0.6215
Epoch: 100, Train Acc: 0.5507, Test Acc: 0.5047, Train Loss: 0.6100, Test Loss: 0.6153
Epoch: 110, Train Acc: 0.7601, Test Acc: 0.7130, Train Loss: 0.5937